## Data Science Capstone Week 3 - Part 3

#### In this notebook, I explore and cluster the neighborhoods in Toronto using the dataframe created in the previous part.

In [1]:
# First import necessary libraries
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
print('pandas and numpy imported...')

import requests # library to handle requests
print('requests imported...')

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('matplotlib imported...')

# import k-means from clustering stage
from sklearn.cluster import KMeans
print('KMeans imported...')

import json # library to handle JSON files
print('json imported...')

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
print('json_normalize imported. \n Done.')


pandas and numpy imported...
requests imported...
matplotlib imported...
KMeans imported...
json imported...
json_normalize imported. 
 Done.


In [2]:

!conda install -c conda-forge folium=0.5.0
import folium # map rendering library
print('folium imported!')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [6]:
# Read the csv file stored in the 'body' variable which contains the data from part 2 and display the first five lines.
df = pd.read_csv(body)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [7]:
# I need to install one more package in order to get the geocoordinates for Toronto (This has already run
# once so it's okay to skip this and the next cell.)
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [8]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [5]:
# To save time and avoid installing geocode every time I log on, I'm going to hard code the "latitude" and "longitude" variables.
latitude = 43.653963
longitude = -79.387207

In [9]:
# Create map of Toronto with the neighborhoods superimposed on it.
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, code in zip(df['Latitude'], df['Longitude'], df['Borough'], df['PostalCode']):
    label = '{}, {}'.format(borough, code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

For this next step, I'm going to just focus on the boroughs in "Downtown Toronto".

In [10]:
# I will take a slice of the dataframe, df, to get just the postal codes in Downtown Toronto.
downtown_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
5,M6G,Downtown Toronto,Christie,43.669542,-79.422564
6,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
7,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
8,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
9,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817


I'll now plot the new subset of points on the map using folium, like we did with all of the Toronto boroughs.

In [11]:
# Create map of Toronto with the neighborhoods superimposed on it.
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, code in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['PostalCode']):
    label = '{}'.format(code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

Next I use Foursquare to explore venues near each postal code zone in downtown Toronto

In [13]:
# Define some parameters for the call to Foursquare
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100

In [14]:
#Define a function that returns a dataframe of nearby venues for a given list of neighborhoods/postal codes
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now I run the above function on each postal code in downtown Toronto and save the venues in a new dataframe called Old_Toronto_Venues

In [15]:
Old_Toronto_Venues = getNearbyVenues(names=downtown_data['PostalCode'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

M5A
M5B
M5C
M5E
M5G
M6G
M5H
M5J
M5K
M5L
M5S
M5T
M5V
M4W
M5W
M4X
M5X
M4Y


In [16]:
print(Old_Toronto_Venues.shape)
Old_Toronto_Venues.head()

(1296, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [17]:
# Saving the venues dataframe so I can pick up from here later
Old_Toronto_Venues.to_csv('Old_Toronto_Venues.csv', index = None)

In [18]:
venues_df=pd.read_csv('Old_Toronto_Venues.csv')
venues_df.head()

,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,M5A,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Check how many venues were returned for each postal code.

In [19]:
venues_df.groupby('PostalCode').count()

,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
M4W,4,4,4,4,4,4
M4X,44,44,44,44,44,44
M4Y,87,87,87,87,87,87
M5A,52,52,52,52,52,52
M5B,100,100,100,100,100,100
M5C,100,100,100,100,100,100
M5E,56,56,56,56,56,56
M5G,86,86,86,86,86,86
M5H,100,100,100,100,100,100


Find out how many unique categories of venues exist in downtown Toronto.

In [22]:
print('There are {} unique categories of venues.'.format(len(venues_df['Venue Category'].unique())))

There are 204 unique categories of venues.


### In this next section I analyze each neighborhood to find the top seven kinds of venues

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Now I group rows by PostalCode and take the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

,PostalCode,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4W,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.25,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
1,M4X,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000
2,M4Y,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,...,0.00,0.00,0.00,0.000000,0.011494,0.00,0.000000,0.011494,0.00,0.011494
3,M5A,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019231,...,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.019231
4,M5B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,...,0.01,0.00,0.00,0.010000,0.010000,0.01,0.010000,0.000000,0.00,0.000000
5,M5C,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.00,0.00,0.00,0.010000,0.000000,0.00,0.010000,0.000000,0.00,0.000000
6,M5E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.017857,0.000000,0.00,0.000000,0.000000,0.00,0.000000
7,M5G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011628,0.000000,...,0.00,0.00,0.00,0.011628,0.000000,0.00,0.011628,0.000000,0.00,0.011628
8,M5H,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00,0.00,0.00,0.010000,0.000000,0.00,0.010000,0.000000,0.01,0.000000
9,M5J,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.01,0.010000,0.000000,0.00,0.010000,0.000000,0.00,0.000000


#### Now I can easily determine the top 7 venue categories for each postal code

In [26]:
num_top_venues = 7

for code in toronto_grouped['PostalCode']:
    print("----"+code+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M4W----
                       venue  freq
0                       Park  0.50
1                 Playground  0.25
2                      Trail  0.25
3          Afghan Restaurant  0.00
4               Neighborhood  0.00
5         Mexican Restaurant  0.00
6  Middle Eastern Restaurant  0.00


----M4X----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.05
2  Chinese Restaurant  0.05
3                Café  0.05
4              Bakery  0.05
5         Pizza Place  0.05
6  Italian Restaurant  0.05


----M4Y----
                      venue  freq
0               Coffee Shop  0.08
1       Japanese Restaurant  0.05
2          Sushi Restaurant  0.05
3                   Gay Bar  0.03
4                Restaurant  0.03
5              Burger Joint  0.03
6  Mediterranean Restaurant  0.02


----M5A----
         venue  freq
0  Coffee Shop  0.17
1          Pub  0.06
2         Park  0.06
3         Café  0.06
4       Bakery  0.06
5   Restaurant  0.04
6      Theater  0.04




In [28]:
# a function to sort the venue categories in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now I make a new dataframe and display the top 10 venues for each neighborhood

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postalcodes_venues_sorted = pd.DataFrame(columns=columns)
postalcodes_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    postalcodes_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

postalcodes_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,M4X,Coffee Shop,Chinese Restaurant,Park,Restaurant,Pub,Italian Restaurant,Café,Bakery,Pizza Place,Breakfast Spot
2,M4Y,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Burger Joint,Gay Bar,Fast Food Restaurant,Dance Studio,Men's Store,Mediterranean Restaurant
3,M5A,Coffee Shop,Café,Pub,Park,Bakery,Restaurant,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market
4,M5B,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store,Ice Cream Shop,Burger Joint


## Clustering

### Now I use k-means to cluster the postal code areas into 5 clusters, based on venue categories

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

Make a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [31]:
# add clustering labels
postalcodes_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = downtown_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(postalcodes_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Café,Pub,Park,Bakery,Restaurant,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store,Ice Cream Shop,Burger Joint
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Hotel,Restaurant,Café,Clothing Store,Cosmetics Shop,Breakfast Spot,Italian Restaurant,Bakery,Gastropub
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Cheese Shop,Steakhouse,Seafood Restaurant,Café,Basketball Stadium
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Gym / Fitness Center,Salad Place,Middle Eastern Restaurant,Japanese Restaurant


### I want to visualize the resulting clusters on a map

In [33]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now I examine each cluster to see what distinguishes it from the others.
### Cluster 0 - Grocery Stores

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,0,Grocery Store,Café,Park,Coffee Shop,Candy Store,Diner,Italian Restaurant,Baby Store,Athletics & Sports,Restaurant


### Cluster 1 - Coffee Shops

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Café,Pub,Park,Bakery,Restaurant,Theater,Breakfast Spot,Mexican Restaurant,Farmers Market
1,Downtown Toronto,1,Clothing Store,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Sporting Goods Shop,Italian Restaurant,Lingerie Store,Ice Cream Shop,Burger Joint
2,Downtown Toronto,1,Coffee Shop,Hotel,Restaurant,Café,Clothing Store,Cosmetics Shop,Breakfast Spot,Italian Restaurant,Bakery,Gastropub
3,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Farmers Market,Cheese Shop,Steakhouse,Seafood Restaurant,Café,Basketball Stadium
4,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Gym / Fitness Center,Salad Place,Middle Eastern Restaurant,Japanese Restaurant
6,Downtown Toronto,1,Coffee Shop,Café,Bar,Thai Restaurant,American Restaurant,Bakery,Steakhouse,Asian Restaurant,Sushi Restaurant,Hotel
7,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Café,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Park,Italian Restaurant
8,Downtown Toronto,1,Coffee Shop,Café,Hotel,American Restaurant,Restaurant,Gastropub,Seafood Restaurant,Bar,Italian Restaurant,Deli / Bodega
9,Downtown Toronto,1,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Steakhouse
14,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Hotel,Beer Bar,Seafood Restaurant,Cocktail Bar,Japanese Restaurant,Bakery


### Cluster 2 - Parks

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


### Cluster 3 - Restaurants

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Downtown Toronto,3,Café,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Restaurant,Bookstore,Poutine Place,Dessert Shop,Beer Bar
11,Downtown Toronto,3,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Café,Chinese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Dessert Shop


### Cluster 4 - Airport

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,4,Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Sculpture Garden,Plane,Boat or Ferry,Coffee Shop,Airport Gate


#### Summary of findings
From the map and cluster analysis above, we clearly have one main cluster in downtown Toronto which is filled with coffer shops, cafe's, bars, and restaurants. It is surrounded by three outliers - parks, the airport, and grocery stores.